<a href="https://colab.research.google.com/github/neginsadat/EA/blob/main/Another_copy_of_untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!apt-get update
!apt-get install -y python3-rna



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import RNA
print("ViennaRNA version:", RNA.__version__)


ViennaRNA version: 2.7.0


In [1]:
!pip uninstall viennarna
!pip install viennarna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 26.4 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import RNA
print("ViennaRNA version:", RNA.__version__)

ViennaRNA version: 2.7.0


In [ ]:
!ls /content/drive/MyDrive/


 46ae9c2a-8058-4cef-b7e1-e613a2c2f89c.gdoc
 chatsettingsbackup.db.crypt1
'Colab Notebooks'
'Google AI Studio'
 SmsContactsBackup
 statusranking.db.crypt1
 stickers.db.crypt1
'To-do list.gsheet'
'Untitled presentation.gslides'
'این رو به فارسی ترجمه کن و جدول آن رو بکش چیز دیگ....gsheet'
'این صفحه رو ترجمه و توضیح بده.gsheet'
'این عکس را  با دقیق  ترین حالت ممکن ترجمه کن.gsheet'
'به نام خدای بخشنده ی مهربان.gdoc'
 تحقیق.gslides
'جدید '
'جلسه اول رباتیک .gdoc'
'دانلود مقاله طراحی سیستمهای مدرن اطلاعاتی و بازشناسی خودكار الگوها.gdoc'
'سند بی‌عنوان (1).gdoc'
'سند بی‌عنوان (2).gdoc'
'سند بی‌عنوان (3).gdoc'
'سند بی‌عنوان.gdoc'
'کپی از Retro Internet Interface Personal Organizer for College by Slidesgo.gslides'


In [9]:
!pip install ViennaRNA


In [14]:
import random
import time
import math
from typing import List, Tuple
import sys
import RNA

###############################################################################
# تنظیمات کلی (بر اساس مقاله و افزودن Local Search)
###############################################################################

POP_SIZE = 52           # اندازه جمعیت m2dRNAs مطابق مقاله
SBX_PROB = 0.9
ETA_C = 10              # SBX
ETA_M = 5               # Mutation
STAGNATION_GEN_ES = 5
MUT_PROB_ES = 0.01      # نرخ جهش ES
TIME_PER_CALL_ES = 2.0
MAX_GENERATION = 10000  # حداکثر نسل در m2dRNAs

# پارامترهای مربوط به جستجوی محلی
LOCAL_SEARCH_ATTEMPTS = 10
LOCAL_SEARCH_RADIUS = 0.05
LOCAL_SEARCH_PROB = 0.4  # احتمال تغییر یک ژن در هر تلاش

###############################################################################
# توابع کمکی: parse_dot_bracket, similarity_db
###############################################################################

def parse_dot_bracket(db: str) -> List[Tuple[int,int]]:
    stack=[]
    pairs=[]
    for i, ch in enumerate(db):
        if ch=='(':
            stack.append(i)
        elif ch==')':
            if stack:
                o= stack.pop()
                pairs.append((o,i))
    pairs.sort(key=lambda x:x[0])
    return pairs

def similarity_db(pred: str, target: str)-> float:
    if len(pred)!= len(target):
        return 0.0
    match_count= sum(1 for i in range(len(pred)) if pred[i]== target[i])
    return match_count/ len(target)

###############################################################################
# PyViennaRNA Wrappers
###############################################################################

def fold_rna(seq: str)-> str:
    fc= RNA.fold_compound(seq)
    ss,mfe= fc.mfe()
    return ss

def partition_function(seq: str)-> float:
    fc= RNA.fold_compound(seq)
    fc.pf()
    ss,mfe= fc.mfe()
    return math.exp(-mfe/0.61633)

def ensemble_diversity(seq: str)-> float:
    fc= RNA.fold_compound(seq)
    fc.pf()
    return fc.mean_bp_distance()

def composition_objective(seq: str)-> float:
    n= len(seq)
    pA= seq.count('A')/n
    pC= seq.count('C')/n
    pG= seq.count('G')/n
    pU= seq.count('U')/n
    mean_= (pA+pC+pG+pU)/4
    var_= ((pA-mean_)**2 +(pC-mean_)**2 +(pG-mean_)**2 +(pU-mean_)**2)/4
    return math.sqrt(var_)

###############################################################################
# decode chromosome
###############################################################################

def decode_chromosome(chrom: List[float], pairs: List[Tuple[int,int]], length_db: int)-> str:
    n_pairs= len(pairs)
    n_unp= length_db - 2*n_pairs
    if n_unp<0:
        n_unp=0
    seq_list= ['A']* length_db
    for i,(a,b) in enumerate(pairs):
        val= chrom[i]
        if val< 0.33:
            seq_list[a], seq_list[b] = 'G','C'
        elif val< 0.66:
            seq_list[a], seq_list[b] = 'A','U'
        else:
            seq_list[a], seq_list[b] = 'G','U'
    up_chrom= chrom[n_pairs:]
    idx_=0
    for i in range(length_db):
        if seq_list[i]=='A':  # پیشفرض
            if idx_< len(up_chrom):
                val= up_chrom[idx_]
                idx_+=1
                if val<0.25:
                    seq_list[i]='A'
                elif val<0.5:
                    seq_list[i]='C'
                elif val<0.75:
                    seq_list[i]='G'
                else:
                    seq_list[i]='U'
    return "".join(seq_list)

###############################################################################
# SBX / Mutation
###############################################################################

def sbx_crossover(p1: List[float], p2: List[float], eta_c: float=ETA_C, pc: float=SBX_PROB):
    if random.random()> pc:
        return p1[:], p2[:]
    c1, c2= p1[:], p2[:]
    size= len(p1)
    for i in range(size):
        if random.random()<0.5:
            if abs(p1[i]-p2[i])>1e-14:
                y1= min(p1[i], p2[i])
                y2= max(p1[i], p2[i])
                rand_= random.random()
                beta= 1.0 + (2.0*y1/(y2-y1))
                alpha= 2.0- pow(beta, -(eta_c+1.0))
                if rand_<=1.0/alpha:
                    betaq= pow(rand_*alpha, 1.0/(eta_c+1.0))
                else:
                    betaq= pow(1.0/(2.0- rand_*alpha),1.0/(eta_c+1.0))
                c1_i= 0.5*((y1+y2)- betaq*(y2-y1))
                beta= 1.0 + (2.0*(1.0-y2)/(y2-y1))
                alpha= 2.0- pow(beta, -(eta_c+1.0))
                if rand_<=1.0/ alpha:
                    betaq= pow(rand_*alpha,1.0/(eta_c+1.0))
                else:
                    betaq= pow(1.0/(2.0- rand_*alpha),1.0/(eta_c+1.0))
                c2_i= 0.5*((y1+y2)+ betaq*(y2-y1))
                c1_i= max(min(c1_i,1.0),0.0)
                c2_i= max(min(c2_i,1.0),0.0)
                if random.random()<0.5:
                    c1[i], c2[i]= c1_i, c2_i
                else:
                    c1[i], c2[i]= c2_i, c1_i
    return c1, c2

def polynomial_mutation(chrom: List[float], pm: float):
    for i in range(len(chrom)):
        if random.random()< pm:
            y= chrom[i]
            delta1= (y-0.0)/(1.0-0.0)
            delta2= (1.0-y)/(1.0-0.0)
            rand_= random.random()
            mut_pow= 1.0/(ETA_M+1.0)
            if rand_<0.5:
                xy= 1.0- delta1
                val= 2.0*rand_ + (1.0-2.0*rand_)*(xy**(ETA_M+1.0))
                deltaq= val**(mut_pow)-1.0
            else:
                xy= 1.0- delta2
                val= 2.0*(1.0-rand_) + 2.0*(rand_-0.5)*(xy**(ETA_M+1.0))
                deltaq= 1.0- (val**(mut_pow))
            y+= deltaq
            y= max(min(y,1.0),0.0)
            chrom[i]= y

###############################################################################
# fast_non_dominated_sort
###############################################################################

def fast_non_dominated_sort(obj_vals: List[List[float]])-> List[List[int]]:
    pop_size= len(obj_vals)
    S= [[] for _ in range(pop_size)]
    n=[0]*pop_size
    fronts= [[]]

    for p in range(pop_size):
        S[p]= []
        n[p]= 0
        for q in range(pop_size):
            p_better= False
            q_better= False
            for k in range(len(obj_vals[p])):
                if obj_vals[p][k]< obj_vals[q][k]:
                    q_better= True
                elif obj_vals[p][k]> obj_vals[q][k]:
                    p_better= True
            if q_better and not p_better:
                n[p]+=1
            elif p_better and not q_better:
                S[p].append(q)
        if n[p]==0:
            fronts[0].append(p)
    i=0
    while len(fronts[i])>0:
        next_front=[]
        for p in fronts[i]:
            for q in S[p]:
                n[q]-=1
                if n[q]==0:
                    next_front.append(q)
        i+=1
        fronts.append(next_front)
    while len(fronts[-1])==0:
        fronts.pop()
        if not fronts:
            break
    return fronts

###############################################################################
# local_search
###############################################################################

def local_search(chrom: List[float], pairs: List[Tuple[int,int]], length_db: int, target_db: str,
                attempts: int=LOCAL_SEARCH_ATTEMPTS,
                radius: float=LOCAL_SEARCH_RADIUS,
                prob_gene: float=LOCAL_SEARCH_PROB)-> List[float]:
    """
    روی کروموزوم ورودی، با ایجاد تغییرات کوچک در ژن‌ها تلاش می‌کند شباهت را بهبود دهد.
    """
    best= chrom[:]
    best_seq= decode_chromosome(best, pairs, length_db)
    best_sim= similarity_db(fold_rna(best_seq), target_db)

    for _ in range(attempts):
        candidate= best[:]
        # تغییر در بخشی از ژن‌ها
        for j in range(len(candidate)):
            if random.random()< prob_gene:
                step= random.gauss(0, radius)
                candidate[j]= min(max(candidate[j]+ step, 0.0),1.0)
        cand_seq= decode_chromosome(candidate, pairs, length_db)
        cand_sim= similarity_db(fold_rna(cand_seq), target_db)
        if cand_sim> best_sim:
            best= candidate
            best_sim= cand_sim
    return best

###############################################################################
# m2dRNAs با Local Search
###############################################################################

def m2dRNAs(sub_db: str, max_time: float)-> Tuple[str,str]:
    """
    نسخه‌ای از m2dRNAs که در هر نسل، روی بهترین فرد جستجوی محلی اعمال می‌کند.
    """
    start_time= time.time()
    pairs= parse_dot_bracket(sub_db)
    length_db= len(sub_db)
    n_pairs= len(pairs)
    n_unp= length_db-2*n_pairs
    if n_unp<0:
        n_unp=0
    chrom_len= n_pairs+ n_unp

    # مقداردهی اولیه
    population= []
    for _ in range(POP_SIZE):
        indiv= [random.random() for __ in range(chrom_len)]
        population.append(indiv)

    def evaluate_population(pop):
        obj_vals=[]
        seq_cache=[]
        for indiv in pop:
            seq_ = decode_chromosome(indiv, pairs, length_db)
            pf= partition_function(seq_)
            ed= ensemble_diversity(seq_)
            co= composition_objective(seq_)
            obj_vals.append([pf, ed, co])
            seq_cache.append(seq_)
        return obj_vals, seq_cache

    generation=0
    while True:
        elapsed= time.time()- start_time
        if elapsed>= max_time:
            break
        if generation> MAX_GENERATION:
            break

        objs, seqs= evaluate_population(population)
        # چک اگر ساختاری کاملاً با sub_db منطبق شد
        for i, sq_ in enumerate(seqs):
            if fold_rna(sq_)== sub_db:
                return (sq_, sub_db)

        # local search روی بهترین فرد (از نظر similarity)
        best_idx=0
        best_sim=0
        for i, sq_ in enumerate(seqs):
            sim_= similarity_db(fold_rna(sq_), sub_db)
            if sim_> best_sim:
                best_sim= sim_
                best_idx= i

        if best_sim<1.0:
            best_chrom= population[best_idx]
            improved_chrom= local_search(best_chrom, pairs, length_db, sub_db)
            imp_seq= decode_chromosome(improved_chrom, pairs, length_db)
            imp_sim= similarity_db(fold_rna(imp_seq), sub_db)
            if imp_sim> best_sim:
                population[best_idx]= improved_chrom

        # مرتب‌سازی غیربابسته
        fronts= fast_non_dominated_sort(objs)
        new_pop= []
        for fr in fronts:
            if len(new_pop)+ len(fr)<= POP_SIZE:
                new_pop.extend([population[x] for x in fr])
            else:
                need= POP_SIZE- len(new_pop)
                random.shuffle(fr)
                new_pop.extend([population[x] for x in fr[:need]])
                break

        # تولید offspring
        pm_value= 1.0/chrom_len  # مطابق مقاله
        offspring=[]
        for i in range(0,POP_SIZE,2):
            p1= new_pop[i]
            p2= new_pop[i+1] if i+1<POP_SIZE else new_pop[0]
            c1, c2= sbx_crossover(p1,p2)
            polynomial_mutation(c1, pm_value)
            polynomial_mutation(c2, pm_value)
            offspring.append(c1)
            offspring.append(c2)
        population= offspring[:POP_SIZE]

        generation+=1

    # در پایان
    objs, seqs= evaluate_population(population)
    best_idx=0
    best_sim=0
    best_seq=None
    for i,indiv in enumerate(population):
        sim_= similarity_db(fold_rna(seqs[i]), sub_db)
        if sim_> best_sim:
            best_sim= sim_
            best_idx= i
    return seqs[best_idx], fold_rna(seqs[best_idx])

###############################################################################
# ساختار decomposition ساده + حل subproblem
###############################################################################

class SubstructureNode:
    def __init__(self, db: str):
        self.db= db
        self.children= []
        self.parent= None
        self.solution_seq= None
        self.solution_db= None
        self.size= len(db)

def build_decomposition(target_db: str, guide: List[int])-> SubstructureNode:
    """
    نسخهٔ ساده‌شده: هر جفت‌باز که guide=1 باشد، آن segment را جدا می‌کنیم.
    """
    root= SubstructureNode(target_db)
    pairs= parse_dot_bracket(target_db)
    for i, g in enumerate(guide):
        if g==1:
            (a,b)= pairs[i]
            # ایجاد یک subproblem شامل فقط آن جفت
            db_stem= list('.'* len(target_db))
            db_stem[a]= '('
            db_stem[b]= ')'
            child_db= "".join(db_stem)
            child_node= SubstructureNode(child_db)
            child_node.parent= root
            root.children.append(child_node)
    return root

def solve_substructure(node: SubstructureNode, time_alloc: float):
    """
    حل یک نود DAG: اگر فرزند ندارد، مستقیم با m2dRNAs حل می‌کنیم
    اگر فرزند دارد، زمان را بر اساس اندازه فرزندان تقسیم می‌کنیم و سپس خود node را هم حل می‌کنیم.
    """
    if not node.children:
        seq, pred= m2dRNAs(node.db, time_alloc)
        node.solution_seq= seq
        node.solution_db= pred
    else:
        total_size= sum(ch.size for ch in node.children)
        remain= time_alloc
        for ch in node.children:
            frac= ch.size/ total_size
            t_ch= frac* time_alloc
            if t_ch> remain:
                t_ch=remain
            solve_substructure(ch, t_ch)
            remain-= t_ch

        # پس از حل فرزندان، node را هم حل می‌کنیم
        # ساده: m2dRNAs خودش
        if remain>0:
            seq, pred= m2dRNAs(node.db, remain)
            node.solution_seq= seq
            node.solution_db= pred
        else:
            node.solution_seq= '.'* node.size
            node.solution_db= '.'* node.size

def final_solution(node: SubstructureNode)-> Tuple[str,str]:
    # نسخه ساده: صرفاً solution_seq خود نود را خروجی می‌دهیم.
    return (node.solution_seq, node.solution_db)

###############################################################################
# eM2dRNAs : نسخه نهایی
###############################################################################

def eM2dRNAs_final(target_db: str, guide: List[int], max_time: float)-> Tuple[str,str]:
    root= build_decomposition(target_db, guide)
    solve_substructure(root, max_time)
    return final_solution(root)

###############################################################################
# ES
###############################################################################

def es_em2dRNAs(target_db: str, total_time: float)-> Tuple[str,str]:
    """
    الگوریتم تکاملی برای راهنما (guide) که decomposition را هدایت می‌کند.
    """
    start_time= time.time()
    pairs= parse_dot_bracket(target_db)
    x= [0]* len(pairs)
    for i in range(len(x)):
        if i%10==0:
            x[i]=1
    es_budget= total_time/2

    seq_best, db_best= eM2dRNAs_final(target_db, x, TIME_PER_CALL_ES)
    best_sim= similarity_db(db_best, target_db)
    pm_es= MUT_PROB_ES
    stagnation=0
    while True:
        elapsed= time.time()- start_time
        if elapsed>= es_budget:
            break
        if best_sim>=1.0:
            break

        def mutate_guide(guide, pm):
            g2= guide[:]
            for i in range(len(g2)):
                if random.random()<pm:
                    g2[i]= 1-g2[i]
            return g2

        y= mutate_guide(x, pm_es)
        seq_y, db_y= eM2dRNAs_final(target_db, y, TIME_PER_CALL_ES)
        sim_y= similarity_db(db_y, target_db)

        if sim_y> best_sim:
            x= y
            seq_best, db_best= seq_y, db_y
            best_sim= sim_y
            pm_es= MUT_PROB_ES
            stagnation=0
        else:
            stagnation+=1
            if stagnation>= STAGNATION_GEN_ES:
                pm_es*=1.2
                stagnation=0

    used= time.time()- start_time
    remain= total_time- used
    if remain>0 and best_sim<1.0:
        seq_last, db_last= eM2dRNAs_final(target_db, x, remain)
        sim_l= similarity_db(db_last, target_db)
        if sim_l> best_sim:
            seq_best, db_best= seq_last, db_last
            best_sim= sim_l

    return seq_best, db_best

###############################################################################
# خواندن دیتاست
###############################################################################

def read_eterna_dataset(filename: str):
    puzzles= []
    with open(filename,'r') as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith("Puzzle #"):
                continue
            fields= line.split('\t')
            if len(fields)<5:
                continue
            puzzle_id= fields[0].strip()
            puzzle_name= fields[1].strip()
            secondary_structure= fields[4].strip()
            puzzles.append((puzzle_id, puzzle_name, secondary_structure))
    return puzzles

###############################################################################
# main
###############################################################################

def main():
    random.seed(0)

    # فرض کنید فایل دیتاست در Colab یا جای دیگر
    dataset_file= "/content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt"


    puzzles= read_eterna_dataset(dataset_file)
    print(f"Read {len(puzzles)} puzzles from {dataset_file}")

    # مثلاً 1 ساعت برای هر پازل
    time_per_puzzle= 3600.0

    # فقط 5 پازل اول
    puzzles= puzzles[:5]

    for pid, pname, pdb in puzzles:
        print(f"\nPuzzle {pid} ({pname}) ...")
        seq, pred= es_em2dRNAs(pdb, time_per_puzzle)
        sim= similarity_db(pred, pdb)
        print(f"  Found seq= {seq}")
        print(f"  Pred DB=  {pred}")
        print(f"  Similarity= {sim:.4f}")

if __name__=="__main__":
    main()


Read 100 puzzles from /content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt

Puzzle 1 (Simple Hairpin) ...
  Found seq= GGGGGGAGCCACUUCC
  Pred DB=  (((((......)))))
  Similarity= 1.0000

Puzzle 2 (Arabidopsis Thaliana 6 RNA - Difficulty Level 0) ...
  Found seq= GGGGGGGUGUAGGGGCCCUCACGGUCCUGGGAGGGGUAGGGGGGGAGGGGGGGGGAGUUGGGAGGCGGAAAUCUCCACUUUCCAUCCCCUCUCUCCCAAUCCUUCCUAAUCCUCCU
  Pred DB=  ((((((((....(((((.....)))))((((((((((.((((((((((((((((((((..(((((......)))))))))))).))))))))))))).))))))))))))))))))
  Similarity= 0.8707

Puzzle 3 (Prion Pseudoknot - Difficulty Level 0) ...
  Found seq= GAAAGGCGGAGUCGACUCUCUUGUUCGGUAAAAAAA
  Pred DB=  ((((((.((((....))))))).)))..........
  Similarity= 1.0000

Puzzle 4 (Human Integrated Adenovirus - Difficulty Level 0) ...
  Found seq= UGGGGGGGGGUGGGUGGGGGGUCAGCUAGGUGAGGAUGGGUGGGAUUGAGGGGGGGGUGGUGGGGAGGGGGGUGGCAGAUUAUUCUUUCUUUACCGUUCCUUCCGACCUUUCUUGUCGUCUUUUAUGAGGGACGGUACGUUCUUCUUUUUAUGCCAAAAAAUUUUCUACUCUAAAACUCCUUAAACCCUAAA
  Pred DB=  (((((

In [ ]:
import random
import time
import math
from typing import List, Tuple
import sys
import RNA

###############################################################################
# تنظیمات کلی
###############################################################################

POP_SIZE = 52              # جمعیت برای m2dRNAs
SBX_PROB = 0.9
ETA_C = 10
ETA_M = 5
STAGNATION_GEN_ES = 5
MUT_PROB_ES = 0.01
TIME_PER_CALL_ES = 2.0
MAX_GEN_NS = 10000        # حداکثر نسل در NSGA-II

WARM_START_SOLUTIONS = 5   # تعداد راه‌حل‌های فاز گرم کردن (warm start)

###############################################################################
# توابع مربوط به ساختار و شباهت
###############################################################################

def parse_dot_bracket(db: str) -> List[Tuple[int,int]]:
    stack=[]
    pairs=[]
    for i,ch in enumerate(db):
        if ch=='(':
            stack.append(i)
        elif ch==')':
            if stack:
                o= stack.pop()
                pairs.append((o,i))
    pairs.sort(key=lambda x:x[0])
    return pairs

def similarity_db(pred: str, target: str)-> float:
    if len(pred)!= len(target):
        return 0.0
    return sum(1 for i in range(len(pred)) if pred[i]==target[i]) / len(target)

###############################################################################
# توابع PyViennaRNA
###############################################################################

def fold_rna(seq: str)-> str:
    fc= RNA.fold_compound(seq)
    ss,mfe= fc.mfe()
    return ss

def partition_function(seq: str)-> float:
    fc= RNA.fold_compound(seq)
    fc.pf()
    ss,mfe= fc.mfe()
    return math.exp(-mfe/0.61633)

def ensemble_diversity(seq: str)-> float:
    fc= RNA.fold_compound(seq)
    fc.pf()
    return fc.mean_bp_distance()

def composition_objective(seq: str)-> float:
    n= len(seq)
    pA= seq.count('A')/n
    pC= seq.count('C')/n
    pG= seq.count('G')/n
    pU= seq.count('U')/n
    mean_= (pA+pC+pG+pU)/4
    var_ = ((pA-mean_)**2 + (pC-mean_)**2 + (pG-mean_)**2 + (pU-mean_)**2)/4
    return math.sqrt(var_)

###############################################################################
# decode / encode کروموزوم
###############################################################################

def decode_chromosome(chrom: List[float], pairs: List[Tuple[int,int]], length_db: int)-> str:
    n_pairs= len(pairs)
    n_unp= length_db -2*n_pairs
    if n_unp<0:
        n_unp=0
    seq_list= ['A']* length_db
    # جفت‌باز
    for i,(a,b) in enumerate(pairs):
        val= chrom[i]
        if val< 0.33:
            seq_list[a], seq_list[b]= 'G','C'
        elif val< 0.66:
            seq_list[a], seq_list[b]= 'A','U'
        else:
            seq_list[a], seq_list[b]= 'G','U'
    # بدون جفت
    up_chrom= chrom[n_pairs:]
    idx_=0
    for i in range(length_db):
        if seq_list[i]=='A':  # پیش‌فرض
            if idx_< len(up_chrom):
                val= up_chrom[idx_]
                idx_+=1
                if val<0.25:
                    seq_list[i]='A'
                elif val<0.5:
                    seq_list[i]='C'
                elif val<0.75:
                    seq_list[i]='G'
                else:
                    seq_list[i]='U'
    return "".join(seq_list)

def encode_chromosome(seq: str, pairs: List[Tuple[int,int]])-> List[float]:
    """
    یک روش ساده برای تبدیل یک توالی به کروموزوم float:
      - جفت GC -> ~0.1, AU -> ~0.5, GU -> ~0.8
      - unpaired: A->0.2, C->0.4, G->0.6, U->0.8
    """
    n_pairs= len(pairs)
    length_db= len(seq)
    chrom_len= n_pairs + (length_db-2*n_pairs)
    result= [0.0]* chrom_len

    # بخش جفت‌باز
    for i,(a,b) in enumerate(pairs):
        pair_ = seq[a]+ seq[b]
        if pair_ in ("GC","CG"):
            result[i]= 0.1
        elif pair_ in ("AU","UA"):
            result[i]= 0.5
        elif pair_ in ("GU","UG"):
            result[i]= 0.8
        else:
            result[i]= 0.0

    used= set()
    for (a,b) in pairs:
        used.add(a)
        used.add(b)

    idx_=0
    up_start= n_pairs
    for i in range(length_db):
        if i not in used:
            if seq[i]=='A':
                result[up_start+ idx_]= 0.2
            elif seq[i]=='C':
                result[up_start+ idx_]= 0.4
            elif seq[i]=='G':
                result[up_start+ idx_]= 0.6
            elif seq[i]=='U':
                result[up_start+ idx_]= 0.8
            else:
                result[up_start+ idx_]= 0.0
            idx_+=1
    return result

###############################################################################
# SBX/ Mutation
###############################################################################

def sbx_crossover(p1: List[float], p2: List[float], eta_c=10, pc=0.9):
    if random.random()>pc:
        return p1[:], p2[:]
    c1, c2= p1[:], p2[:]
    size= len(p1)
    for i in range(size):
        if random.random()<0.5:
            if abs(p1[i]-p2[i])>1e-14:
                y1= min(p1[i], p2[i])
                y2= max(p1[i], p2[i])
                rand_= random.random()
                beta= 1.0 + (2.0*y1/(y2-y1))
                alpha= 2.0- pow(beta, -(eta_c+1.0))
                if rand_<=1.0/ alpha:
                    betaq= pow(rand_*alpha, 1.0/(eta_c+1.0))
                else:
                    betaq= pow(1.0/(2.0- rand_*alpha),1.0/(eta_c+1.0))
                c1_i= 0.5*((y1+y2)- betaq*(y2-y1))

                beta= 1.0 + (2.0*(1.0-y2)/(y2-y1))
                alpha= 2.0- pow(beta, -(eta_c+1.0))
                if rand_<=1.0/alpha:
                    betaq= pow(rand_*alpha,1.0/(eta_c+1.0))
                else:
                    betaq= pow(1.0/(2.0- rand_*alpha),1.0/(eta_c+1.0))
                c2_i= 0.5*((y1+y2)+ betaq*(y2-y1))
                c1_i= max(min(c1_i,1.0),0.0)
                c2_i= max(min(c2_i,1.0),0.0)
                if random.random()<0.5:
                    c1[i], c2[i]= c1_i,c2_i
                else:
                    c1[i], c2[i]= c2_i,c1_i
    return c1,c2

def polynomial_mutation(chrom: List[float], pm: float):
    for i in range(len(chrom)):
        if random.random()< pm:
            y= chrom[i]
            delta1= y
            delta2= 1.0-y
            rand_= random.random()
            mut_pow= 1.0/(ETA_M+1.0)
            if rand_<0.5:
                xy= 1.0- delta1
                val= 2.0*rand_ + (1.0 -2.0*rand_)* (xy**(ETA_M+1.0))
                deltaq= val**(mut_pow)-1.0
            else:
                xy= 1.0- delta2
                val= 2.0*(1.0-rand_) +2.0*(rand_-0.5)*(xy**(ETA_M+1.0))
                deltaq= 1.0- (val**(mut_pow))
            y+= deltaq
            if y<0.0: y=0.0
            if y>1.0: y=1.0
            chrom[i]= y

def fast_non_dominated_sort(obj_vals: List[List[float]])-> List[List[int]]:
    pop_size= len(obj_vals)
    S= [[] for _ in range(pop_size)]
    n=[0]*pop_size
    fronts= [[]]

    for p in range(pop_size):
        S[p]= []
        n[p]=0
        for q in range(pop_size):
            p_better= False
            q_better= False
            for k in range(len(obj_vals[p])):
                if obj_vals[p][k]< obj_vals[q][k]:
                    q_better=True
                elif obj_vals[p][k]> obj_vals[q][k]:
                    p_better=True
            if q_better and not p_better:
                n[p]+=1
            elif p_better and not q_better:
                S[p].append(q)
        if n[p]==0:
            fronts[0].append(p)
    i=0
    while len(fronts[i])>0:
        next_front=[]
        for p in fronts[i]:
            for q in S[p]:
                n[q]-=1
                if n[q]==0:
                    next_front.append(q)
        i+=1
        fronts.append(next_front)
    while len(fronts[-1])==0:
        fronts.pop()
        if not fronts:
            break
    return fronts

###############################################################################
# Warm Start: ساخت k راه‌حل اولیه ساده
###############################################################################

def warm_start_solutions(db: str, k: int)-> List[List[float]]:
    pairs= parse_dot_bracket(db)
    length_db= len(db)
    solutions=[]
    for _ in range(k):
        seq_list= list('.'* length_db)
        for (a,b) in pairs:
            # ساده: همه جفت‌بازها GC
            seq_list[a], seq_list[b]= 'G','C'
        for i in range(length_db):
            if seq_list[i]=='.':
                seq_list[i]= random.choice(['A','C','G','U'])
        seq_str= "".join(seq_list)
        chrom= encode_chromosome(seq_str, pairs)
        solutions.append(chrom)
    return solutions

###############################################################################
# m2dRNAs با warm start
###############################################################################

def m2dRNAs_warm(db: str, max_time: float)-> Tuple[str,str]:
    start_time= time.time()
    pairs= parse_dot_bracket(db)
    length_db= len(db)
    n_pairs= len(pairs)
    n_unp= length_db- 2*n_pairs
    if n_unp<0:
        n_unp=0
    chrom_len= n_pairs+ n_unp

    # ساخت جمعیت با گرم کردن
    warm_list= warm_start_solutions(db, WARM_START_SOLUTIONS)
    population=[]
    # وارد کردن راه‌حل‌های warm
    for c in warm_list:
        population.append(c)
    # تکمیل جمعیت
    need= POP_SIZE- len(population)
    for _ in range(need):
        indiv= [random.random() for __ in range(chrom_len)]
        population.append(indiv)

    def evaluate_population(pop):
        obj_vals=[]
        seq_cache=[]
        for indiv in pop:
            seq_= decode_chromosome(indiv, pairs, length_db)
            pf= partition_function(seq_)
            ed= ensemble_diversity(seq_)
            co= composition_objective(seq_)
            obj_vals.append([pf, ed, co])
            seq_cache.append(seq_)
        return obj_vals, seq_cache

    generation=0
    while True:
        elapsed= time.time()- start_time
        if elapsed>= max_time:
            break
        if generation> MAX_GEN_NS:
            break

        objs, seqs= evaluate_population(population)
        # check similarity=1
        for i, sq_ in enumerate(seqs):
            if fold_rna(sq_)== db:
                return (sq_, db)

        fronts= fast_non_dominated_sort(objs)
        new_pop=[]
        for fr in fronts:
            if len(new_pop)+ len(fr)<= POP_SIZE:
                new_pop.extend([population[x] for x in fr])
            else:
                need2= POP_SIZE- len(new_pop)
                random.shuffle(fr)
                new_pop.extend([population[x] for x in fr[:need2]])
                break

        pm_val= 1.0/chrom_len
        offspring=[]
        for i in range(0,POP_SIZE,2):
            p1= new_pop[i]
            p2= new_pop[i+1] if i+1<POP_SIZE else new_pop[0]
            c1,c2= sbx_crossover(p1,p2)
            polynomial_mutation(c1, pm_val)
            polynomial_mutation(c2, pm_val)
            offspring.append(c1)
            offspring.append(c2)
        population= offspring[:POP_SIZE]
        generation+=1

    # best
    objs, seqs= evaluate_population(population)
    best_idx=0
    best_sim=0
    for i,indiv in enumerate(population):
        sim_= similarity_db(fold_rna(seqs[i]), db)
        if sim_> best_sim:
            best_sim= sim_
            best_idx= i
    return (seqs[best_idx], fold_rna(seqs[best_idx]))

###############################################################################
# Decomposition + حل Substructure
###############################################################################

class SubstructureNode:
    def __init__(self, db: str):
        self.db= db
        self.children= []
        self.parent= None
        self.solution_seq= None
        self.solution_db= None
        self.size= len(db)

def build_decomposition(target_db: str, guide: List[int])-> SubstructureNode:
    root= SubstructureNode(target_db)
    pairs= parse_dot_bracket(target_db)
    for i,g in enumerate(guide):
        if g==1:
            (a,b)= pairs[i]
            db_stem= list('.'* len(target_db))
            db_stem[a], db_stem[b]= '(', ')'
            child_db= "".join(db_stem)
            child_node= SubstructureNode(child_db)
            child_node.parent= root
            root.children.append(child_node)
    return root

def solve_substructure(node: SubstructureNode, time_alloc: float):
    if not node.children:
        # حل با m2dRNAs_warm
        seq, pred= m2dRNAs_warm(node.db, time_alloc)
        node.solution_seq= seq
        node.solution_db= pred
    else:
        total_size= sum(ch.size for ch in node.children)
        remain= time_alloc
        for ch in node.children:
            frac= ch.size / total_size
            t_ch= frac* time_alloc
            if t_ch> remain:
                t_ch= remain
            solve_substructure(ch, t_ch)
            remain-= t_ch
        if remain>0:
            seq, pred= m2dRNAs_warm(node.db, remain)
            node.solution_seq= seq
            node.solution_db= pred
        else:
            node.solution_seq= '.'* node.size
            node.solution_db= '.'* node.size

def final_solution(node: SubstructureNode)-> Tuple[str,str]:
    return (node.solution_seq, node.solution_db)

def eM2dRNAs_final(target_db: str, guide: List[int], max_time: float)-> Tuple[str,str]:
    root= build_decomposition(target_db, guide)
    solve_substructure(root, max_time)
    return final_solution(root)

###############################################################################
# ES
###############################################################################

def es_em2dRNAs(target_db: str, total_time: float)-> Tuple[str,str]:
    """
    Evolutionary Strategy برای راهنما
    """
    start_time= time.time()
    pairs= parse_dot_bracket(target_db)
    x= [0]* len(pairs)
    for i in range(len(x)):
        if i%10==0:
            x[i]=1
    es_budget= total_time/2

    seq_best, db_best= eM2dRNAs_final(target_db, x, TIME_PER_CALL_ES)
    best_sim= similarity_db(db_best, target_db)
    pm_es= MUT_PROB_ES
    stagnation=0

    while True:
        elapsed= time.time()- start_time
        if elapsed>= es_budget:
            break
        if best_sim>=1.0:
            break

        def mutate_guide(guide, pm):
            g2= guide[:]
            for i in range(len(g2)):
                if random.random()<pm:
                    g2[i]= 1-g2[i]
            return g2

        y= mutate_guide(x, pm_es)
        seq_y, db_y= eM2dRNAs_final(target_db, y, TIME_PER_CALL_ES)
        sim_y= similarity_db(db_y, target_db)

        if sim_y> best_sim:
            x= y
            seq_best, db_best= seq_y, db_y
            best_sim= sim_y
            pm_es= MUT_PROB_ES
            stagnation=0
        else:
            stagnation+=1
            if stagnation>= STAGNATION_GEN_ES:
                pm_es*=1.2
                stagnation=0

    used= time.time()- start_time
    remain= total_time- used
    if remain>0 and best_sim<1.0:
        seq_last, db_last= eM2dRNAs_final(target_db, x, remain)
        sim_l= similarity_db(db_last, target_db)
        if sim_l> best_sim:
            seq_best, db_best= seq_last, db_last
            best_sim= sim_l
    return (seq_best, db_best)

###############################################################################
# خواندن دیتاست Eterna100
###############################################################################

def read_eterna_dataset(filename: str):
    puzzles= []
    with open(filename,'r') as f:
        for line in f:
            line=line.strip()
            if not line or line.startswith("Puzzle #"):
                continue
            fields= line.split('\t')
            if len(fields)<5:
                continue
            puzzle_id= fields[0].strip()
            puzzle_name= fields[1].strip()
            secondary_structure= fields[4].strip()
            puzzles.append((puzzle_id, puzzle_name, secondary_structure))
    return puzzles

###############################################################################
# main
###############################################################################

def main():
    random.seed(0)

    dataset_file= "/content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt"
    puzzles= read_eterna_dataset(dataset_file)
    print(f"Read {len(puzzles)} puzzles from {dataset_file}")

    # هر پازل چقدر زمان؟
    time_per_puzzle= 600.0  # به عنوان نمونه 10 دقیقه

    # به عنوان تست، فقط 5 پازل اول:
    puzzles= puzzles[5:15]

    for pid, pname, pdb in puzzles:
        print(f"\nPuzzle {pid} ({pname}) ...")
        seq, pred= es_em2dRNAs(pdb, time_per_puzzle)
        sim= similarity_db(pred, pdb)
        print(f"  Found seq= {seq}")
        print(f"  Pred DB = {pred}")
        print(f"  Similarity= {sim:.4f}")

if __name__=="__main__":
    main()


Read 100 puzzles from /content/drive/MyDrive/Colab Notebooks/eterna100_vienna2.txt

Puzzle 6 (Saccharomyces Cerevisiae - Difficulty Level 0) ...
